In [1]:
import numpy as np
import cvxpy as cp 
from queue import Queue
from copy import deepcopy
from time import sleep
from BnB_normal import BnBNormalAlgorythm
import pandas as pd

In [2]:
A = np.matrix([[1, 2, 3],[3, 4, 3],[2, 1, 0]])
B = np.matrix([[1, 2, 3],[3, 4, 3],[2, 1, 0]])
x = np.array([1,0,0.5])
y = np.matrix([3,4,5]).T
np.linalg.norm(x)
np.count_nonzero(x)
lambda_0 = 1
M0 = 10000
A @ B

matrix([[13, 13,  9],
        [21, 25, 21],
        [ 5,  8,  9]])

In [3]:
A @ x

matrix([[2.5, 4.5, 2. ]])

# We want to solve the following

$$\min_{x \in \real^n} \frac{1}{2} ||y - A x||^2_2 + \lambda ||x||_0$$

In [4]:
def evaluation_main (y, A, x, alpha = lambda_0):
    value = 0.5 * np.linalg.norm( y - A @ x ) ** 2 + alpha * np.count_nonzero(x)    
    return value

In [5]:
evaluation_main(y, A, x)

14.749999999999998

# We are going to create a BnB of the following

$$\min_{x \in \real^n} \frac{1}{2} ||y - A x||^2_2 + \frac{\lambda}{M} ||x_{\bar{S}}||_1 + \lambda |S_1|$$

In [6]:
np.random.seed(421)
A = np.random.randint(-10, 10, (10, 10))
y = np.random.randint(-10, 10, (10, 1))
n = y.shape[0]
n

10

In [7]:
lambda_0 = 1
M0 = 100


def evaluation_main (y, A, x, alpha = lambda_0):
    value = 0.5 * np.linalg.norm( y - A @ x ) ** 2 + alpha * np.count_nonzero(x)    
    return value


def check_bound(u, v):
    if v.pu <= u.pu:
        return v.pu, v
    else:
        return u.pu, u
    



def relax_problem (y, A, S0, S1, S, alpha = lambda_0, M = M0):
    n = y.shape[0]
    X = cp.Variable((n, 1))
    obj =  0.5 * cp.sum_squares(y - A @ X ) 
    obj +=  alpha / M * cp.norm(X[S], 1)
    obj += alpha * len(S1)

    
    if len(S0) == 0:
        constraint = [X <= M, -X <= M] 
    else:
        constraint = [X[S0] == 0, X <= M, -X <= M] 

    prob = cp.Problem(cp.Minimize( obj ), constraint)
    result = prob.solve(verbose = False, solver = cp.ECOS, abstol=1e-4, reltol=1e-4)
    
    #eliminate 0
    x_vector = X.value
    x_vector = np.where(abs(x_vector) < 1e-7, 0, x_vector )
    return result, x_vector



class Node:
    
    def __init__(self, n, S0, S1, S, A, y):
        self.level = n - len(S)
        self.A = A
        self.y = y
        self.S0 = S0
        self.S1 = S1
        self.S = S
        self.pvl = 0
        self.pu = None       #corresponds to obj func value in real problem with x
        self.x = None
        self.calculate_pvl()
        self.calculate_pu_x()
        
    
    def calculate_pvl (self) -> float:
        self.pvl, self.x = relax_problem(y, A, S0 = self.S0, S1 = self.S1, S = self.S)
        return self.pvl
        
    def calculate_pu_x (self) -> float:
        self.pu =  evaluation_main(y, A, self.x)
        return self.pu
        

    def __le__ (self, other) -> bool:
        return self.pvl <= other.pvl
    
    def __str__(self) -> str:
        text = f"S0: {self.S0} | S1: {self.S1} | S: {self.S}\nPvl: {self.pvl}\n\n"
        return text
    
    def __repr__(self) -> str:
        text = f"S0: {self.S0} | S1: {self.S1} | Pvl: {self.pvl}"
        return text

In [8]:
S = list(range(n))
S0 = []
S1 = []
u = Node(n, S0, S1, S, A, y)

In [9]:
S = list(range(n))
S0 = []
S1 = []

u1 = Node(n, S0, S1, S, A, y)
print(u1.x)
print(u1.calculate_pvl())
print(u1.calculate_pu_x())

S = deepcopy(u.S)
S1 = deepcopy(u.S1)
S0 = deepcopy(u.S0)

index = S.pop(0)
index2 = S.pop(0)

#None zero branch
S0.append(index)
S0.append(index2)

u2 = Node(n, S0, S1, S, A, y)

print(u1 >= u2)
print(u1.pu, u2.pu)
check_bound(u1, u2)

[[ 0.23170507]
 [-0.43967887]
 [-0.81020599]
 [ 0.62298317]
 [ 0.09508656]
 [ 0.45371026]
 [-0.08893891]
 [ 0.82366556]
 [-0.06056742]
 [-1.4987601 ]]
0.051354165675013726
10.000101146445013
False
10.000101146445013 22.80588833660041


(10.000101146445013, S0: [] | S1: [] | Pvl: 0.051354165675013726)

## We are going to create the order

In [10]:
S = list(range(n))
S0 = []
S1 = []

#crear queue
q = Queue()
u = Node(n, S0, S1, S, A, y)
q.put(u)

#optimum 
pv_opt = u.pu
node_opt = deepcopy(u)
print("first op:", pv_opt)




while not q.empty():
    
    u = q.get()
    print(f"u node: S0: {u.S0} | S1: {u.S1} | Pvl: {u.pvl}")
    
    #if the node is leaf
    if u.level == n  or len(u.S) == 0:
        continue
    
    S_v = deepcopy(u.S)
    S1_v = deepcopy(u.S1)
    S0_v = deepcopy(u.S0)
    
    index = S_v.pop(0)
    
    #None zero branch
    S1_v.append(index)
    
    v1 = Node(n, S0_v, S1_v, S_v, A, y)
    print("v1 pvl: ", v1.pvl)
    if not pv_opt <= v1.pvl:
        q.put(v1)
        print(v1.pu)             
        pv_opt, node_opt = check_bound(node_opt, v1)

        
    
    #Zero branch
    S_v = deepcopy(u.S)
    S1_v = deepcopy(u.S1)
    S0_v = deepcopy(u.S0)
    index = S_v.pop(0)
    S0_v.append(index)
    v0 = Node(n, S0_v, S1_v, S_v, A, y)
    print("V0 pvl:", v0.pvl)
    if not pv_opt <= v0.pvl:
        q.put(v0)
        print(v0.pu)             
        pv_opt, node_opt = check_bound(node_opt, v0)
    
    print("Queue:\t",list(q.queue), "\n")

    

print("Opt obj value:\t", pv_opt)
print("Optimum node:\n", node_opt ) 

first op: 10.000101146445013
u node: S0: [] | S1: [] | Pvl: 0.051354165675013726
v1 pvl:  1.0490305868541316
10.000063112047783
V0 pvl: 0.25192521160595616
9.207988171831724
Queue:	 [S0: [] | S1: [0] | Pvl: 1.0490305868541316, S0: [0] | S1: [] | Pvl: 0.25192521160595616] 

u node: S0: [] | S1: [0] | Pvl: 1.0490305868541316
v1 pvl:  2.044623951208538
10.000047815949076
V0 pvl: 3.137170806392232
11.099604311844722
Queue:	 [S0: [0] | S1: [] | Pvl: 0.25192521160595616, S0: [] | S1: [0, 1] | Pvl: 2.044623951208538, S0: [1] | S1: [0] | Pvl: 3.137170806392232] 

u node: S0: [0] | S1: [] | Pvl: 0.25192521160595616
v1 pvl:  1.2488451784407324
9.20798866013061
V0 pvl: 14.854662245513893
Queue:	 [S0: [] | S1: [0, 1] | Pvl: 2.044623951208538, S0: [1] | S1: [0] | Pvl: 3.137170806392232, S0: [0] | S1: [1] | Pvl: 1.2488451784407324] 

u node: S0: [] | S1: [0, 1] | Pvl: 2.044623951208538
v1 pvl:  3.0365114934775246
10.000028746516655
V0 pvl: 8.2354751769116
15.205751148022989
Queue:	 [S0: [1] | S1: [0

V0 pvl: 7.24403960969072
14.213992599904488
Queue:	 [S0: [0] | S1: [1] | Pvl: 1.2488451784407324, S0: [] | S1: [0, 1, 2] | Pvl: 3.0365114934775246, S0: [2] | S1: [0, 1] | Pvl: 8.2354751769116, S0: [1] | S1: [0, 2] | Pvl: 4.131656120475425, S0: [1, 2] | S1: [0] | Pvl: 7.24403960969072] 

u node: S0: [0] | S1: [1] | Pvl: 1.2488451784407324
v1 pvl:  2.241620846789978
9.207979700484703
V0 pvl: 8.955488283338461
15.930518324081504
Queue:	 [S0: [] | S1: [0, 1, 2] | Pvl: 3.0365114934775246, S0: [2] | S1: [0, 1] | Pvl: 8.2354751769116, S0: [1] | S1: [0, 2] | Pvl: 4.131656120475425, S0: [1, 2] | S1: [0] | Pvl: 7.24403960969072, S0: [0] | S1: [1, 2] | Pvl: 2.241620846789978, S0: [0, 2] | S1: [1] | Pvl: 8.955488283338461] 

u node: S0: [] | S1: [0, 1, 2] | Pvl: 3.0365114934775246
v1 pvl:  4.030268578879637
10.000020520248443
V0 pvl: 9.669539783003579
Queue:	 [S0: [2] | S1: [0, 1] | Pvl: 8.2354751769116, S0: [1] | S1: [0, 2] | Pvl: 4.131656120475425, S0: [1, 2] | S1: [0] | Pvl: 7.24403960969072, S

In [11]:
np.random.seed(420)
n = 15
A = np.random.randint(-10, 10, (n, n))
y = np.random.randint(-10, 10, (n, 1))

lambda_0 = 1
M0 = 100

In [12]:
node = node_opt
node

S0: [0, 4, 6] | S1: [1, 2, 3, 5, 7, 8, 9] | Pvl: 7.510651296502245

In [13]:
u = node.y -  node.A @ node.x
node.A[: , 1 ] @ u

array([4.60092871e-06])

In [14]:
for i in range(5, -1, -1):
    print(i)

5
4
3
2
1
0


In [15]:
np.random.seed(42)
n = 8
A = np.random.randint(-10, 10, (n, n))
y = np.random.randint(-10, 10, (n, 1))

lambda_0 = 1
M0 = 100

solver = BnBNormalAlgorythm(y, A, lambda0=lambda_0, M=M0)

pv, node, num_nodes, nodes = solver.solve(verbose=False)

print("Solution Node")
# remember to count node 0
print(f"Nodes visited {num_nodes}/{2**(n) + 1}")
print(node)

print(node.x)

print("Getting lambda intervals")
node.get_lambda_interval(verbose = True)

Initializing normal solver BnB
.

............

            ############################################################
            	Solver finished in:	4.183932542800903 s
            ############################################################

Solution Node
Nodes visited 99/257
S0: [0, 1, 2, 4, 5, 6] | S1: [3, 7] | S: []
Pvl: 2.9164648947133163

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.90412079]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.49853023]]
Getting lambda intervals
[]
omega: [0.00091997]
J0: -1
Leaf Node
[ -inf , inf ]


(-inf, inf)

In [21]:
lower_bounds = []
upper_bounds = []
for node in nodes:
    lower_bound, upper_bound = node.get_lambda_interval(verbose = True)
    lower_bounds.append(lower_bound)
    upper_bounds.append(upper_bound)

[val : 8.0368, idx : 3, val : 8.0147, idx : 4, val : 7.9996, idx : 6, val : 7.9982, idx : 5, val : 7.9944, idx : 2, val : 7.9942, idx : 0, val : 7.9786, idx : 1, val : 7.9594, idx : 7]
omega: -1.0893734510346533
J0: 7
[ -inf , 7.959414487867278 ]
J: 7
[ -inf , 7.959414487867278 ]
J: 7
[ -inf , 7.959414487867278 ]
J: 6
First interval [ 7.959414487867278 , 7.959414487867278 ]
J: 5
First interval [ 7.978609577146223 , 7.959414487867278 ]
J: 4
First interval [ 7.994220670880381 , 7.959414487867278 ]
J: 3
First interval [ 7.994355615739934 , 7.959414487867278 ]
J: 2
First interval [ 7.998150417654415 , 7.959414487867278 ]
J: 1
First interval [ 7.999584265269988 , 7.959414487867278 ]
J: 0
[ 8.036808727505473 , inf ]
[ -inf , inf ]
[val : 8.1699, idx : 4, val : 8.1188, idx : 3, val : 8.01, idx : 2, val : 7.9606, idx : 6, val : 7.9594, idx : 5, val : 7.7391, idx : 1, val : 7.7009, idx : 7]
omega: [-1.02906472]
J0: 6
[ -inf , 7.70090419600038 ]
J: 6
[ -inf , 7.70090419600038 ]
J: 6
[ -inf , 7.7

In [20]:
lower_bounds

[-inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf,
 -inf]

In [19]:
highest_lower_bound = max(lower_bounds)
lowest_higher_bound = min(upper_bounds)
print("[", highest_lower_bound, ",", lowest_higher_bound, "]")

[ -inf , inf ]
